In [86]:
import netket as nk
import numpy as np
import matplotlib.pyplot as plt
import json

from netket import experimental as nkx
import netket.nn as nknn
import netket.optimizer as nkopt
import jax
import jax.numpy as jnp

In [130]:
# create graph

x = 2  # take a 2x4 lattice
y = 4

t = 1  # tunneling/hopping
U = 0.01  # coulomb

# create the graph our fermions can hop on
g = nk.graph.Grid(extent=[x, y], pbc=False)
n_sites = g.n_nodes

g

Lattice(
    n_nodes=8,
    extent=[2 4],
    basis_vectors=
        [[1. 0.]
         [0. 1.]],
    site_offsets=
        [[0. 0.]],
)

In [131]:
# create an operator representing fermi hubbard interactions
# -t (i^ j + h.c.) + U (i^ i j^ j)
# we will create a helper function to abbreviate the creation, destruction and number operators
# each operator has a site and spin projection (sz) in order to find the right position in the hilbert space samples
def c(site):
    return nkx.operator.fermion.destroy(hi, site)
    
def cdag(site):
    return nkx.operator.fermion.create(hi, site)

def nc(site):
    return nkx.operator.fermion.number(hi, site)

In [154]:
ham = 0.0
for u, v in g.edges():
    ham += -t * (cdag(u) * c(v) + cdag(v) * c(u))

    ham += U * nc(u) * nc(v)

In [155]:
print("Hamiltonian =", ham.operator_string())

Hamiltonian = -1.0 [0^ 1] +
-1.0 [1^ 0] +
0.01 [0^ 0 1^ 1] +
-1.0 [1^ 2] +
-1.0 [2^ 1] +
0.01 [1^ 1 2^ 2] +
-1.0 [0^ 4] +
-1.0 [4^ 0] +
0.01 [0^ 0 4^ 4] +
-1.0 [1^ 5] +
-1.0 [5^ 1] +
0.01 [1^ 1 5^ 5] +
-1.0 [3^ 7] +
-1.0 [7^ 3] +
0.01 [3^ 3 7^ 7] +
-1.0 [2^ 3] +
-1.0 [3^ 2] +
0.01 [2^ 2 3^ 3] +
-1.0 [6^ 7] +
-1.0 [7^ 6] +
0.01 [6^ 6 7^ 7] +
-1.0 [4^ 5] +
-1.0 [5^ 4] +
0.01 [4^ 4 5^ 5] +
-1.0 [2^ 6] +
-1.0 [6^ 2] +
0.01 [2^ 2 6^ 6] +
-1.0 [5^ 6] +
-1.0 [6^ 5] +
0.01 [5^ 5 6^ 6]


In [156]:
hi = nkx.hilbert.SpinOrbitalFermions(n_sites, n_fermions=8)
hi

SpinOrbitalFermions(n_orbitals=8, n_fermions=8)

In [157]:
occupation = np.zeros(n_sites)
occupation[0] = occupation[1] = 1
occupation[2] = occupation[3] = 1
occupation[4] = occupation[5] = 1
occupation[6] = occupation[7] = 1
#occupation /= np.linalg.norm(occupation)
res = ham.get_conn(occupation)
res

(array([[1., 1., 1., 1., 1., 1., 1., 1.]]), array([0.1]))

In [158]:
Hdense = ham.to_dense()
i = hi.states_to_numbers(occupation)
v = np.zeros(hi.n_states)
v[i] = 1

v /= np.linalg.norm(v)

In [159]:
E = (v.conj().T).dot(Hdense.dot(v))
E

0.09999999999999999